In [1]:
%matplotlib notebook

In [2]:
from pyk4a import PyK4A, Config, Calibration
import numpy as np
from copy import deepcopy
import pyk4a
from matplotlib import pyplot as plt
from matplotlib.pyplot import subplots
import matplotlib.animation as animation
from IPython.display import HTML
import json

In [3]:
import numpy as np
import pyk4a
from pyk4a import Config, PyK4A, Calibration, CalibrationType
config = Config(depth_mode=pyk4a.DepthMode.WFOV_2X2BINNED,
                    synchronized_images_only=True)
k4adev = PyK4A(config=config)
calib = Calibration(device = k4adev,
                    config=config,
                    source_calibration=CalibrationType.COLOR,
                    target_calibration=CalibrationType.COLOR)
k4adev.connect()

In [5]:
# Get the next color frame without the depth (blocking function)
img_color, img_depth = k4adev.get_capture(color_only=False, transform_depth_to_color=True)
img_color.shape, img_depth.shape

((720, 1280, 4), (720, 1280))

In [6]:
#pix2d = [539, 325]
#depth = img_depth[pix2d[0]][pix2d[1]]
#calib.convert_2d_to_3d(pix2d, depth) # , CalibrationType.COLOR, CalibrationType.COLOR)
calib.convert_2d_to_3d([500,300], 235)# , CalibrationType.COLOR, CalibrationType.COLOR)

(1, [-54.37434387207031, -26.070945739746094, 235.0])

In [7]:
calib.convert_2d_to_3d(pix2d, depth)  # , source_camera=CalibrationType.COLOR, target_camera=CalibrationType.COLOR)

NameError: name 'pix2d' is not defined

In [8]:
calib.convert_3d_to_3d([10,20,30], CalibrationType.COLOR, CalibrationType.DEPTH)

[42.12046813964844, 19.063861846923828, 27.87849998474121]

In [9]:
# Display with pyplot
fig, ax = subplots(figsize=(16/2, 9/2))
im_depth = plt.imshow(img_depth, cmap='gray')
im_color = plt.imshow(img_color[:, :, :3][...,::-1])    # BGRA to RGB for matplotlib
ims = []
ims.append([im_depth]) # , im_color]
ims.append([im_color])
ani = animation.ArtistAnimation(fig, ims, interval=4000, blit=True)     # , repeat_delay=1000)
HTML(ani.to_jshtml())
plt.show()
# HTML(ani.to_html5_video())

<IPython.core.display.Javascript object>

In [ ]:
from math import floor
fig, ax = subplots(figsize=(16/2, 9/2))
#im_depth = plt.imshow(img_color[:, :, :3][...,::-1]) # , cmap='gray')
img_color, img_depth = k4a.get_capture(color_only=False, transform_depth_to_color=True)
im_depth = plt.imshow(img_depth, cmap='gray')
text=ax.text(0, .05, "Depth : ", va="bottom", ha="left")

def onclick(event):
    x = floor(event.xdata)
    y = floor(event.ydata)
#     tx = floor(event.xdata)
    if img_depth[x, y]:
        tx = 'x=%d, y=%d, depth =%d' % (x, y, img_depth[x, y])
    else:
        tx = 'x=%d, y=%d, depth = NO VALUE' % (x, y)
    text.set_text(tx)
    print(tx)

cid = fig.canvas.mpl_connect('button_press_event', onclick)
plt.show()

In [4]:
k4adev.disconnect()